<a href="https://colab.research.google.com/github/NicoPozio/WildfireDetectionDL/blob/main/notebook/Classifier_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NicoPozio/WildfireDetectionDL.git

Cloning into 'WildfireDetectionDL'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 193 (delta 4), reused 0 (delta 0), pack-reused 188 (from 1)
Receiving objects: 100% (193/193), 121.46 KiB | 994.00 KiB/s, done.
Resolving deltas: 100% (99/99), done.


In [2]:
import sys

repo_path = '/content/WildfireDetectionDL'
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [3]:
# Cell: Update Code
import os

REPO_PATH = "/content/WildfireDetectionDL"

print("🔄 Syncing with GitHub...")
!cd {REPO_PATH} && git pull

print("Code updated. You can now run the Sweep or Train cell immediately.")

🔄 Syncing with GitHub...
Already up to date.
Code updated. You can now run the Sweep or Train cell immediately.


In [4]:
#Install Project Dependencies
#hydra-core: Configuration management
#wandb: Experiment tracking
#omegaconf: Dict handling for Hydra
!pip install -q hydra-core wandb omegaconf


from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
    print("Google Drive Mounted")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 4.4 MB/s eta 0:00:00
Mounted at /content/drive
Google Drive Mounted


In [5]:
import wandb
from google.colab import userdata
import os

try:
    #Fetch key from Colab Secrets
    api_key = userdata.get('WANDB_API_KEY')

    #Set as Environment Variable
    #This ensures Hydra and subprocesses can find it automatically
    os.environ["WANDB_API_KEY"] = api_key

    wandb.login()
    print("Logged in to WandB")

except Exception as e:
    print(f"Authentication Failed: {e}")
    print("Action Required: Go to the 'Secrets' tab (Key icon) on the left.")
    print("Add a new secret named 'WANDB_API_KEY' with your key from https://wandb.ai/authorize")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: costantini572 (costantini572-sapienza-universit-di-roma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Logged in to WandB


First we download the dataset, we check if the user has the data in its google drive, if not it's downloaded from a public link

In [11]:
import os
import gdown
from src.utils import extract_zip

# Cartella di destinazione su Colab
local_root = "/content/data"
os.makedirs(local_root, exist_ok=True)
!rm -rf {local_root}/*

# Percorsi nel Drive Personale (Piano A)
drive_dataset_path = "/content/drive/MyDrive/Wildfire_Project/dataset.zip"
drive_synthetic_path = "/content/drive/MyDrive/Wildfire_Project/synthetic_wildfire_2k.zip"

# Percorsi finali su Colab
target_dataset_zip = os.path.join(local_root, "dataset.zip")
target_synthetic_zip = os.path.join(local_root, "synthetic_wildfire_2k.zip")

# ---------------------------------------------------------
# LOGICA DI DOWNLOAD
# ---------------------------------------------------------

# 1. Controllo Piano A (Hai fatto la Fase 1?)
if os.path.isfile(drive_dataset_path) and os.path.isfile(drive_synthetic_path):
    print("✅ File trovati nel tuo Drive personale! Copio quelli...")
    !cp "{drive_dataset_path}" "{target_dataset_zip}"
    !cp "{drive_synthetic_path}" "{target_synthetic_zip}"

else:
    print("⚠️ File non trovati nel tuo Drive. Avvio Piano B (Backup Pubblico)...")

    # --- SOLUZIONE ERRORE 'FolderContentsMaximumLimitError' ---
    # Invece di scaricare la cartella intera (che ha troppi file),
    # scarichiamo direttamente i due file ZIP usando i loro ID univoci.

    # INSERISCI QUI GLI ID REALI DEI FILE DALLA CARTELLA CONDIVISA
    DATASET_ID = "17KPBVodZkmBYqz7252mDk6hfY55eMU-Q"
    SYNTHETIC_ID = "1KyI09FDCAkLp1BYO-VgD7zRMrtrl3anK"

    print(f"⬇️ Scarico dataset.zip (ID: {DATASET_ID})...")
    gdown.download(id=DATASET_ID, output=target_dataset_zip, quiet=False)

    print(f"⬇️ Scarico synthetic.zip (ID: {SYNTHETIC_ID})...")
    gdown.download(id=SYNTHETIC_ID, output=target_synthetic_zip, quiet=False)

# 2. Estrazione
print(f"\n📦 Estrazione file in {local_root}...")

if os.path.exists(target_dataset_zip) and os.path.exists(target_synthetic_zip):
    # Verifica che i file non siano corrotti (dimensione > 0)
    if os.path.getsize(target_dataset_zip) > 0:
        extract_zip(target_dataset_zip, local_root)
    else:
        print("❌ Errore: dataset.zip è vuoto o non scaricato.")

    if os.path.getsize(target_synthetic_zip) > 0:
        extract_zip(target_synthetic_zip, local_root)
    else:
        print("❌ Errore: synthetic.zip è vuoto o non scaricato.")

    print("🎉 Dati pronti!")
else:
    print("❌ Errore critico: I file zip mancano. Controlla gli ID.")

⚠️ File non trovati nel tuo Drive. Avvio Piano B (Backup Pubblico)...
⬇️ Scarico dataset.zip (ID: 17KPBVodZkmBYqz7252mDk6hfY55eMU-Q)...


Downloading...
From (original): https://drive.google.com/uc?id=17KPBVodZkmBYqz7252mDk6hfY55eMU-Q
From (redirected): https://drive.google.com/uc?id=17KPBVodZkmBYqz7252mDk6hfY55eMU-Q&confirm=t&uuid=7a19a572-8d3c-4504-8f95-53b5da8a5414
To: /content/data/dataset.zip
100%|██████████| 1.55G/1.55G [00:21<00:00, 73.0MB/s]


⬇️ Scarico synthetic.zip (ID: 1KyI09FDCAkLp1BYO-VgD7zRMrtrl3anK)...


Downloading...
From (original): https://drive.google.com/uc?id=1KyI09FDCAkLp1BYO-VgD7zRMrtrl3anK
From (redirected): https://drive.google.com/uc?id=1KyI09FDCAkLp1BYO-VgD7zRMrtrl3anK&confirm=t&uuid=9e94dea8-dc2f-4fb6-b29d-2a0413b64b4e
To: /content/data/synthetic_wildfire_2k.zip
100%|██████████| 256M/256M [00:04<00:00, 58.7MB/s]



📦 Estrazione file in /content/data...
Extracting dataset.zip to /content/data...


Unzipping: 100%|██████████| 42860/42860 [00:18<00:00, 2380.66files/s]


 -> Success: Extracted to /content/data
Extracting synthetic_wildfire_2k.zip to /content/data...


Unzipping: 100%|██████████| 2003/2003 [00:03<00:00, 545.15files/s] 

 -> Success: Extracted to /content/data
🎉 Dati pronti!


In [12]:
import os
from PIL import Image
from tqdm import tqdm

# Define your data root
DATA_ROOT = "/content/data"

print(f"Scanning {DATA_ROOT} for corrupt images...")

corrupt_count = 0
for root, dirs, files in os.walk(DATA_ROOT):
    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(root, filename)
            try:
                # Try to fully load the image bytes
                with Image.open(file_path) as img:
                    img.load()
            except OSError:
                print(f"Found corrupt image: {file_path}")
                try:
                    os.remove(file_path)
                    print(f"Deleted {filename}")
                    corrupt_count += 1
                except:
                    print(f"Could not delete {filename}")

print(f"\nScan Complete. Removed {corrupt_count} corrupt files.")

Scanning /content/data for corrupt images...
Found corrupt image: /content/data/dataset/train/nowildfire/-114.152378,51.027198.jpg
Deleted -114.152378,51.027198.jpg
Found corrupt image: /content/data/dataset/test/wildfire/-73.15884,46.38819.jpg
Deleted -73.15884,46.38819.jpg

Scan Complete. Removed 2 corrupt files.


In [ ]:
import subprocess
import os

# 1. Define Paths Constants
REPO_ROOT = "/content/WildfireDetectionDL"
SWEEP_CONFIG_PATH = os.path.join(REPO_ROOT, "conf", "sweep.yaml")

# 2. Validation
if not os.path.exists(SWEEP_CONFIG_PATH):
    print(f"CRITICAL ERROR: Could not find sweep.yaml at {SWEEP_CONFIG_PATH}")
else:
    print(f"--- Registering Sweep from {SWEEP_CONFIG_PATH} ---")

    # 3. Register the sweep
    result = subprocess.run(
        ["wandb", "sweep", SWEEP_CONFIG_PATH],
        capture_output=True,
        text=True,
        # CORRECTION: cwd must be the directory, not the file
        cwd=REPO_ROOT
    )

    output_text = result.stderr + result.stdout
    print("Raw Output:", output_text)

    # 4. Extract Sweep ID
    sweep_id = None
    for line in output_text.split('\n'):
        if "wandb agent" in line:
            parts = line.strip().split("wandb agent ")
            if len(parts) > 1:
                sweep_id = parts[-1].strip()
                break

    # 5. Launch the Agent
    if sweep_id:
        print(f"\nSUCCESS: Detected Sweep ID: {sweep_id}")
        print("Starting Agent... (This will run multiple experiments)")

        # CORRECTION: We use the explicit REPO_ROOT variable defined at the top
        !cd {REPO_ROOT} && wandb agent {sweep_id} --count 20
    else:
        print("\nERROR: Could not find Sweep ID.")

--- Registering Sweep from /content/WildfireDetectionDL/conf/sweep.yaml ---
Raw Output: wandb: Creating sweep from: /content/WildfireDetectionDL/conf/sweep.yaml
wandb: Creating sweep with ID: edt8s2ap
wandb: View sweep at: https://wandb.ai/costantini572-sapienza-universit-di-roma/WildfireDetectionDL/sweeps/edt8s2ap
wandb: Run sweep agent with: wandb agent costantini572-sapienza-universit-di-roma/WildfireDetectionDL/edt8s2ap


SUCCESS: Detected Sweep ID: costantini572-sapienza-universit-di-roma/WildfireDetectionDL/edt8s2ap
Starting Agent... (This will run multiple experiments)
wandb: Starting wandb agent 🕵️
2025-11-25 14:31:59,608 - wandb.wandb_agent - INFO - Running runs: []
2025-11-25 14:31:59,991 - wandb.wandb_agent - INFO - Agent received command: run
2025-11-25 14:31:59,991 - wandb.wandb_agent - INFO - Agent starting run with config:
	dataset.augmentation.rotation_degrees: 0
	dataset.params.use_synthetic: False
	model.dropout: 0.3
	model.name: resnet50
	training.batch_size: 64
	tra

In [1]:
import wandb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# ---------------------------------------------------------
# 1. Configuration
# ---------------------------------------------------------
# Based on your logs: https://wandb.ai/pozioniccolo-sapienza-universit-di-roma/WildfireDetectionDL/sweeps/7hmucehp

# REPLACE THESE WITH YOUR EXACT DETAILS FROM THE LOGS
ENTITY = "pozioniccolo-sapienza-universit-di-roma"
PROJECT = "WildfireDetectionDL"  # <--- Correction: Case Sensitive!
SWEEP_ID = "7hmucehp"            # <--- Correction: Hardcoded from your logs to be safe

# Construct the full path
sweep_path = f"{ENTITY}/{PROJECT}/{SWEEP_ID}"
print(f"Connecting to: {sweep_path}...")

# ---------------------------------------------------------
# 2. Fetch Data
# ---------------------------------------------------------
api = wandb.Api()

try:
    sweep = api.sweep(sweep_path)
    print("✅ Connection Successful! Downloading runs...")

    # Get runs sorted by accuracy
    runs = sorted(sweep.runs, key=lambda run: run.summary.get("val_acc", 0), reverse=True)

    results = []
    for run in runs:
        # Only process runs that finished and have metrics
        if "val_acc" in run.summary:
            cfg = run.config

            # Helper to safely get nested keys
            def get_param(config_dict, key_path):
                keys = key_path.split('.')
                val = config_dict
                for k in keys:
                    if isinstance(val, dict):
                        val = val.get(k)
                    else:
                        return None
                return val

            # Extract data using the safe helper
            results.append({
                "run_id": run.id,
                "model": get_param(cfg, 'model.name'),
                "use_synthetic": get_param(cfg, 'dataset.params.use_synthetic'),
                "lr": get_param(cfg, 'training.learning_rate'),
                "batch_size": get_param(cfg, 'training.batch_size'),
                "optimizer": get_param(cfg, 'training.optimizer'),
                "val_acc": run.summary.get("val_acc")
            })

    df = pd.DataFrame(results)

    # ---------------------------------------------------------
    # 3. Analysis & Visualization
    # ---------------------------------------------------------
    if not df.empty:
        best_run = df.iloc[0]
        print("\n🏆 CHAMPION CONFIGURATION 🏆")
        print(f"Run ID:         {best_run['run_id']}")
        print(f"Model:          {best_run['model']}")
        print(f"Synthetic Data: {best_run['use_synthetic']}")
        print(f"Optimizer:      {best_run['optimizer']}")
        print(f"Learning Rate:  {best_run['lr']}")
        print(f"Val Accuracy:   {best_run['val_acc']:.2f}%")

        # Plot
        plt.figure(figsize=(10, 6))
        sns.barplot(data=df, x="model", y="val_acc", hue="use_synthetic", errorbar=None)
        plt.title("Model Performance: Real vs Synthetic Data")
        plt.ylim(80, 100) # Zoom in to the top range (since your acc is high)
        plt.ylabel("Validation Accuracy (%)")
        plt.xlabel("Model Architecture")
        plt.legend(title="Use Synthetic Data")
        plt.grid(axis='y', alpha=0.3)
        plt.show()

    else:
        print("⚠️ No finished runs found with validation accuracy.")

except Exception as e:
    print(f"❌ Error connecting to WandB: {e}")
    print("Check if Entity, Project, or Sweep ID are correct.")

Connecting to: pozioniccolo-sapienza-universit-di-roma/WildfireDetectionDL/7hmucehp...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:



KeyboardInterrupt: 

In [ ]:
import wandb
import os

# 1. Configuration (Must match your logs exactly)
ENTITY = "pozioniccolo-sapienza-universit-di-roma"
PROJECT = "WildfireDetectionDL"

api = wandb.Api()

if 'best_run' in locals() and not df.empty:
    # Get the ID from the pandas row
    run_id = best_run['run_id']

    print(f"🏆 Preparing to test Best Model: {best_run['model']} (Run ID: {run_id})...")

    # 2. Construct the path using known variables
    # Format: entity/project/run_id
    run_path = f"{ENTITY}/{PROJECT}/{run_id}"

    print(f"⬇️ Downloading weights from: {run_path}")

    try:
        # 3. Download 'best_model.pth'
        run_obj = api.run(run_path)
        run_obj.file("best_model.pth").download(replace=True, root=".")

        print("✅ Download Complete. Weights are ready.")

        # 4. Run the Test
        print("🚀 Starting Test Script...")

        # Extract params for the command line
        use_syn = str(best_run['use_synthetic']).lower()
        batch_size = int(best_run['batch_size'])

        !python ./WildfireDetectionDL/test.py \
          model.name={best_run['model']} \
          dataset.params.use_synthetic={use_syn} \
          training.batch_size={batch_size} \
          wandb.mode="online"

    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("Cannot run test: No best run identified in the analysis step.")